# Searching for best models by tuning params

## Libraries

In [1]:
import pandas as pd
import numpy as np
import sklearn 
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
np.random.seed(123)

## Reading processed data (script 'data-preprocessing')

In [2]:
data = pd.read_csv('../processed_data/out.csv')

## Dividing into train and test

To divide set so in both parts with have simmilar amounts of big and small credits we need to put them into groups:  
(in traditional splits, randomization makes results appear very uneven)

In [3]:
from sklearn import preprocessing

x = data[['credit_amount']].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
data['amount_groups'] = x_scaled
bins = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
data['amount_groups'] = np.digitize(data['amount_groups'], bins)
unique, counts = np.unique(data['amount_groups'], return_counts=True)
dict(zip(unique, counts))

{1: 445, 2: 293, 3: 97, 4: 80, 5: 38, 6: 19, 7: 14, 8: 8, 9: 6}

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(['customer_type', 'amount_groups'], axis=1), data.customer_type, test_size=0.20, stratify = data[['amount_groups', 'customer_type']])

## Needed functions

### Encoding

In [5]:
import category_encoders as ce
class Error(Exception):
    pass
class NonMatchingLengthsError(Error):
    pass

In [6]:
def multiEnc(X_train, X_test, target_train, cols, encodings):
    """
    Lista znaków do "encodings":
        d - backward difference
        n - base N
        b - binary
        c - cat boost
        # - hashing
        h - helmert
        j - James-Stein
        l - leave one out
        m - m-estimate
        1 - one-hot
        o - ordinal
        p - polynomial
        s - sum coding
        t - target encoding
        w - weight of evidence
    """
    ce_map = {"d": ce.backward_difference.BackwardDifferenceEncoder,
             "n": ce.basen.BaseNEncoder,
             "b": ce.binary.BinaryEncoder,
             "c": ce.cat_boost.CatBoostEncoder,
             "#": ce.hashing.HashingEncoder,
             "h": ce.helmert.HelmertEncoder,
             "j": ce.james_stein.JamesSteinEncoder,
             "l": ce.leave_one_out.LeaveOneOutEncoder,
             "m": ce.m_estimate.MEstimateEncoder,
             "1": ce.one_hot.OneHotEncoder,
             "o": ce.ordinal.OrdinalEncoder,
             "p": ce.polynomial.PolynomialEncoder,
             "s": ce.sum_coding.SumEncoder,
             "t": ce.target_encoder.TargetEncoder,
             "w": ce.woe.WOEEncoder}
    try:
        if len(cols)!=len(encodings):
            raise(NonMatchingLengthsError)
    except NonMatchingLengthsError:
        print("Lengths do not match")
        return None
    e=0
    for c in cols:
        if X_train[c].dtypes=='object':
            enc=ce_map[encodings[e]](cols=c)
            enc=enc.fit(X_train, target_train)
            X_train=enc.transform(X_train)
            X_test=enc.transform(X_test)
        e=e+1
    return (X_train, X_test)

### Models creation

In [7]:
from abc import (ABC, 
                 abstractmethod, 
                 abstractproperty)
from typing import Any
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier # Inna paczka niż sklearn!
import xgboost as xgb


class Builder(ABC):

    @abstractproperty
    def product(self) -> None:
        pass
    
    
class Product():

    def __init__(self) -> None:
        self.parts = []

    def add(self, part: Any) -> None:
        self.parts.append(part)

    def list_parts(self):
        return self.parts


class ConcreteBuilder(Builder):

    def __init__(self) -> None:

        self.reset()

    def reset(self) -> None:
        self._product = Product()

    @property
    def product(self) -> Product:

        product = self._product
        self.reset()
        return product

    def _add_model(self, model_type: str, params: dict):
        if model_type == 'logistic regression':
            self._logistic_regression(params)
        if model_type == 'decision tree':
            self._decision_tree(params)
        if model_type == 'svm':
            self._svm(params)
        if model_type == 'naive bayes':
            self._naive_bayes(params)
        if model_type == 'random forest':
            self._random_forest(params)
        if model_type == 'ada boost':
            self._ada_boost(params)
        if model_type == 'gradient boost':
            self._gradient_boost(params)
        if model_type == 'xgboost':
            self._xgboost(params)
        
    def _logistic_regression(self, params: dict):
        
        lr = LogisticRegression(**params)
        
        return self._product.add(lr)
    
    def _decision_tree(self, params: dict):
        
        dt = DecisionTreeClassifier(**params)
        
        dt_params = {'criterion': dt.criterion}
        
        return self._product.add(dt)

    def _svm(self, params: dict):
        svm = SVC(**params)
        
        return self._product.add(svm)
    
    def _naive_bayes(self, params: dict):
        nb = GaussianNB(**params)

        return self._product.add(nb)
    
    def _random_forest(self, params: dict):
        
        rf = RandomForestClassifier(**params)
        return self._product.add(rf)
        
    def _ada_boost(self, params: dict):
        ada = AdaBoostClassifier(**params)
    
        return self._product.add(ada)
    
    def _gradient_boost(self, params: dict):
        gb= GradientBoostingClassifier(**params)
        return self._product.add(gb)
    
    def _xgboost(self, params: dict):
        
        xg=XGBClassifier(**params)
        return self._product.add(xg)

class Director:

    def __init__(self) -> None:
        self._builder = None

    @property
    def builder(self) -> Builder:
        return self._builder

    @builder.setter
    def builder(self, builder: Builder) -> None:

        self._builder = builder


    def add_model(self, model_type, params):
        return self.builder._add_model(model_type, params)

    def add_all_models(self):
        self.add_model('logistic regression', {})
        self.add_model('decision tree', {})
        self.add_model('svm', {})
        self.add_model('naive bayes', {})
        self.add_model('random forest', {})
        self.add_model('ada boost', {})
        self.add_model('gradient boost', {})
        self.add_model('xgboost', {})
        return builder.product.list_parts()
    
    def get_all_models(self, metric_name: str = ''):
        parts =  builder.product.list_parts()
        return parts

In [8]:
director = Director()
builder = ConcreteBuilder()
director.builder = builder

### Business metrics to rate models (simmilar to F1, but more focused on earnings for the bank)

In [9]:
ir_loan = 0.13
lgd = 0.38

In [10]:
def calculateEarningsLosses(X_test, y_pred, y_test):
    '''
    As declared, takes test data and predicted classes and calculates:
    - earnings made by following prediction
    - losses made by following prediction
    - earnings omited by following prediction
    - losses omited by following prediction
    '''
    amounts = X_test['credit_amount']
    balance_all = y_test.apply(lambda x: ir_loan if x==1 else -lgd) * amounts
    earnings_made = balance_all.iloc[np.logical_and(y_test==1, y_pred==1).array].sum()
    earnings_omitted = balance_all.iloc[np.logical_and(y_test==1, y_pred==0).array].sum()
    losses_made = balance_all.iloc[np.logical_and(y_test==0, y_pred==1).array].sum()
    losses_omitted = balance_all.iloc[np.logical_and(y_test==0, y_pred==0).array].sum()
    
    results = pd.DataFrame(columns=['Earnings made', 'Earnings omitted', 'Losses made', 'Losses omitted'])
    results.loc[0] = [earnings_made, earnings_omitted, losses_made, losses_omitted]
    
    final_balance = earnings_made + losses_made
    max_income = balance_all.iloc[(y_test==1).array].sum()
    perc_of_max_income = final_balance/max_income
    return (results, final_balance, perc_of_max_income)

## Search for best type of encoding

In [11]:
columns_enc = ['checking_account_status', 'credit_history', 'purpose', 'savings', 'present_employment', 
              'personal', 'other_debtors', 'property', 'other_installment_plans', 
              'housing', 'job', 'telephone']
def encoding_list_gen(nominal, ordinal):
    enc = [ordinal, nominal, nominal, ordinal, nominal, nominal, nominal, nominal, nominal, nominal, nominal, nominal]
    return enc
# we group columns as ordinal and nominal to encode them with the same type
#enc = ['j', '1', '1', 'j', '1', '1', '1', '1', '1', '1', '1', '1']

In [12]:
X_test.columns

Index(['checking_account_status', 'duration', 'credit_history', 'purpose',
       'credit_amount', 'savings', 'present_employment', 'installment_rate',
       'personal', 'other_debtors', 'present_residence', 'property', 'age',
       'other_installment_plans', 'housing', 'existing_credits', 'job',
       'dependents', 'telephone', 'foreign_worker', 'sex',
       'checking_account_exists', 'savings_account_exists'],
      dtype='object')

Previous attempts have shown, that XGB, GradientBoosting, RandomForest and SVM are the classifiers to aim at. In following cells we will evaluate those three algorithms with different encoding types. Because most of them are tree based, OneHot will be used only to test SVM.

### Testing function

In [13]:
from typing import List
from sklearn.metrics import f1_score

def compare_models(models_list: List, X_train, y_train, X_test, y_test, categorical_variables, encoding_list):
    results = dict()
    
    df_train, df_test = multiEnc(X_train, X_test, y_train, categorical_variables, encoding_list)
    
    
    for model in models_list:
        
        training = model.fit(df_train, y_train)
        score = training.score(df_test, y_test)
        f1 = f1_score(y_test, model.predict(df_test))
        _, _, business = calculateEarningsLosses(X_test, model.predict(df_test), y_test)
        results[model] = (['score:', score], ['f1:', f1], ['business:', business])
        
    return results

In [14]:
def compare_encoders(model, X_train, y_train, X_test, y_test, columns_enc, enc_nominal, enc_ordinal):
    results = pd.DataFrame(columns = enc_nominal, index = enc_ordinal)
    results_f1 = pd.DataFrame(columns = enc_nominal, index = enc_ordinal)
    results_bus = pd.DataFrame(columns = enc_nominal, index = enc_ordinal)
    
    for nom in enc_nominal:
        for ordi in enc_ordinal:
            encoding_list = encoding_list_gen(nom, ordi)
            df_train, df_test = multiEnc(X_train, X_test, y_train, columns_enc, encoding_list)
            training = model.fit(df_train, y_train)
            score = training.score(df_test, y_test)
            f1 = f1_score(y_test, model.predict(df_test))
            _, _, business = calculateEarningsLosses(X_test, model.predict(df_test), y_test)
            
            results.loc[ordi, nom] = score
            results_f1.loc[ordi, nom] = f1
            results_bus.loc[ordi, nom] = business
    
    return (results, results_f1, results_bus)

In [15]:
director.add_model('gradient boost', {})
director.add_model('xgboost', {})
director.add_model('random forest', {})
director.add_model('svm', {})

In [16]:
models = director.get_all_models()

In [17]:
enc_nominal = ['l', 'j', 'm'] # encoding types to check for nominal values
enc_ordinal = ['p', 'h', 'd', 'l', 'j'] # encoding types to check for ordinal values
# we don't check any encodings leading to information loss

### Gradient boosting encoders fit

In [18]:
(score, f1, buss) = compare_encoders(models[0], X_train, y_train, X_test, y_test, columns_enc, enc_nominal, enc_ordinal)

In [19]:
score

,l,j,m
p,0.76,0.76,0.76
h,0.745,0.745,0.745
d,0.77,0.77,0.775
l,0.75,0.75,0.75
j,0.75,0.75,0.75


In [20]:
f1

,l,j,m
p,0.833333,0.833333,0.833333
h,0.8223,0.8223,0.8223
d,0.840278,0.840278,0.843206
l,0.827586,0.827586,0.827586
j,0.827586,0.827586,0.827586


In [21]:
buss

,l,j,m
p,0.205109,0.205109,0.205109
h,0.12244,0.12244,0.12244
d,0.109999,0.109999,0.188288
l,0.0454018,0.0454018,0.0454018
j,0.0454018,0.0454018,0.0454018


### XGB encoders fit

In [22]:
(score, f1, buss) = compare_encoders(models[1], X_train, y_train, X_test, y_test, columns_enc, enc_nominal, enc_ordinal = ['l', 'j'])

XGB has some issues with ordinal encoders so we omit them

In [23]:
score

,l,j,m
l,0.71,0.71,0.71
j,0.71,0.71,0.71


In [24]:
f1

,l,j,m
l,0.8,0.8,0.8
j,0.8,0.8,0.8


In [25]:
buss

,l,j,m
l,-0.30158,-0.30158,-0.30158
j,-0.30158,-0.30158,-0.30158


### Random Forest encoders fit

In [26]:
(score, f1, buss) = compare_encoders(models[2], X_train, y_train, X_test, y_test, columns_enc, enc_nominal, enc_ordinal)

In [27]:
score

,l,j,m
p,0.74,0.745,0.72
h,0.755,0.75,0.75
d,0.765,0.75,0.77
l,0.75,0.715,0.725
j,0.745,0.74,0.74


In [28]:
f1

,l,j,m
p,0.821918,0.825939,0.812081
h,0.832765,0.829932,0.832215
d,0.840678,0.828767,0.842466
l,0.829932,0.804124,0.813559
j,0.824742,0.82069,0.824324


In [29]:
buss

,l,j,m
p,0.026208,0.0106196,-0.158493
h,0.129899,-0.0433056,-0.0763733
d,0.0774394,0.106196,0.185287
l,0.072924,-0.0781775,-0.169518
j,0.0105949,0.0400962,-0.105141


### SVM encoders fit

In [30]:
enc_nominal.append('o')

In [31]:
(score, f1, buss) = compare_encoders(models[3], X_train, y_train, X_test, y_test, columns_enc, enc_nominal, enc_ordinal)

In [32]:
score

,l,j,m,o
p,0.71,0.71,0.71,0.71
h,0.71,0.71,0.71,0.71
d,0.71,0.71,0.71,0.71
l,0.71,0.71,0.71,0.71
j,0.71,0.71,0.71,0.71


In [33]:
f1

,l,j,m,o
p,0.826347,0.826347,0.826347,0.826347
h,0.826347,0.826347,0.826347,0.826347
d,0.826347,0.826347,0.826347,0.826347
l,0.826347,0.826347,0.826347,0.826347
j,0.826347,0.826347,0.826347,0.826347


In [34]:
buss

,l,j,m,o
p,-0.400685,-0.400685,-0.400685,-0.400685
h,-0.400685,-0.400685,-0.400685,-0.400685
d,-0.400685,-0.400685,-0.400685,-0.400685
l,-0.400685,-0.400685,-0.400685,-0.400685
j,-0.400685,-0.400685,-0.400685,-0.400685


### Results

According to score and f1 score all of the algorithms perform well. However most important aspect for potenital client is how much money it would make them. In that case only two algorithms have some potential in improving which are:  
**Gradient boosting** with encoding: ordinal: **polynominal** encoding, nominal: **James-Stein** (all performed the same, I just picked that one)  
**Random Forest** with encoding: ordinal: **backward difference**, nominal: **m-estimate**  
Rest algorithms present very low income levels or even lossess.

## Tuning parameters for selected algorithms

In [35]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

Just scoring for bussiness

In [36]:
def bussScore(y, y_pred, X):
    amounts = X['credit_amount']
    balance_all = y.apply(lambda x: ir_loan if x==1 else -lgd) * amounts
    earnings_made = balance_all.iloc[np.logical_and(y==1, y_pred==1).array].sum()
    losses_made = balance_all.iloc[np.logical_and(y==0, y_pred==1).array].sum()
    
    final_balance = earnings_made + losses_made
    max_income = balance_all.iloc[(y==1).array].sum()
    perc_of_max_income = final_balance/max_income
    return perc_of_max_income

Making our own scorer for parameter tuning

In [37]:
b_scorer = make_scorer(bussScore, greater_is_better=True)

Setting encodings

In [38]:
gboost_enc = encoding_list_gen('j', 'p')
rforest_enc = encoding_list_gen('m', 'b')

### Gradient boosting tuning

In [39]:
parameters = {
    "loss":["deviance"],
    "learning_rate": [0.1, 0.15, 0.2],
    "min_samples_split": [0.01, 0.03, 0.05],
    "min_samples_leaf": [0.01, 0.02, 0.03],
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.8, 0.85, 0.9, 1.0],
    "n_estimators":[10, 100]
    }

In [53]:
df_train, df_test = multiEnc(X_train, X_test, y_train, columns_enc, gboost_enc)

In [41]:
grid = GridSearchCV(estimator=models[0], param_grid = parameters, scoring = 'f1', cv=4, n_jobs=-1)
grid_result = grid.fit(df_train, y_train)

In [42]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.868532 using {'criterion': 'friedman_mse', 'learning_rate': 0.15, 'loss': 'deviance', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 0.02, 'min_samples_split': 0.05, 'n_estimators': 10, 'subsample': 0.9}


These are the best parameters to be used for Grid Search. Let's check their bussiness scorer.

In [43]:
best_model_gboost = grid_result.best_estimator_
best_model_gboost.score(df_test, y_test)

0.755

In [45]:
bussScore(y_test, best_model_gboost.predict(df_test), df_test)

-0.1784704402553549

### Random Forest tuning

In [46]:
parameters = {
    "min_samples_split": [0.01, 0.03, 0.05],
    "min_samples_leaf": [0.01, 0.02, 0.03],
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["gini",  "mae"],
    "n_estimators":[10, 100, 150],
    "ccp_alpha":[0.0, 0.01, 0.1]
    }

In [47]:
df_train, df_test = multiEnc(X_train, X_test, y_train, columns_enc, rforest_enc)

In [48]:
grid = GridSearchCV(estimator=models[2], param_grid = parameters, scoring = 'f1', cv=4, n_jobs=-1)
grid_result = grid.fit(df_train, y_train)

In [49]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.854381 using {'ccp_alpha': 0.0, 'criterion': 'gini', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 0.01, 'min_samples_split': 0.01, 'n_estimators': 150}


In [50]:
best_model_rforest = grid_result.best_estimator_
best_model_rforest.score(df_test, y_test)

0.75

In [52]:
bussScore(y_test, best_model_rforest.predict(df_test), df_test)

-0.18053123559755158

## Saving results

In [55]:
def save_final_data(df_train, y_train, df_test, y_test):
    df_train.to_csv('../final_data/df_train.csv', index=False)
    df_test.to_csv('../final_data/df_test.csv', index=False)
    y_test.to_csv('../final_data/y_test.csv', index=False)
    y_train.to_csv('../final_data/y_train.csv', index=False)

save_final_data(df_train, y_train, df_test, y_test)

In [56]:
import pickle

def save_model(model, filename: str):
    pickle.dump(model, open('../models/' + str(filename) + '.sav', 'wb'))

In [57]:
save_model(best_model_gboost, 'gradient_boost_best')

In [58]:
save_model(best_model_rforest, 'random_forest_best')